In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr 
import netCDF4 as nc
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# Idealized wind

In [2]:
def sforc(gpath, vname, fname_out, lat_out, lon_out, time_out, f_out):
    # Save output - this could be a function earlier in the code
    writing = nc.Dataset(gpath + vname + fname_out, "w", format="NETCDF3_64BIT_OFFSET")
    time    = writing.createDimension("time", None)  # sets unlimited dimension
    yh      = writing.createDimension("yh", len(lat_out))
    xh      = writing.createDimension("xh", len(lon_out))

    yh     = writing.createVariable("yh","f8",("yh"))
    yh[:]  = lat_out
    yh.standard_name    = "latitude"
    yh.long_name        = "latitude"
    yh.units            = "degrees_north"
    yh.axis             = "Y"

    xh     = writing.createVariable("xh","f8",("xh"))
    xh[:]  = lon_out
    xh.standard_name    = "longitude"
    xh.long_name        = "longitude"
    xh.units            = "degrees_east"
    xh.axis             = "X" ;

    time    = writing.createVariable("time","f8",("time"))
    time[:] = time_out
    time.standard_name   = "time"
    time.long_name       = "time"
    time.units           = "days since 1900-01-01 00:00:00" ;
    time.calendar        = "gregorian" ;
    time.axis            = "T" ;
    time.modulo          = " " ;

    var          = writing.createVariable(vname,"f8",("time","yh","xh"),fill_value=fillval)
    var[:,:,:]   = f_out
    
    if (vname == 'uas'):
        var.long_name      = "Eastward Near-Surface Wind"
        var.units          = "m s-1"
        
    elif (vname == 'vas'):
        var.long_name      = "Northward Near-Surface Wind"
        var.units          = "m s-1"
    
    writing.close()
    
def save_wind(ds_all, forcings, time_m, lat_out, lon_out, gpath, fname_out):
    
    for iforc, i_ds in zip(forcings,range(len(forcings))):

        pN = ds_all[i_ds]
        time_out  = time_m  # using day of year
        fname_out = fnameLF_out

        # Create a zero array to store the forcing files
        f_out = np.zeros((len(time_out), len(lat_out), len(lon_out)))

        # At every time step 
        for itime in np.arange(0,len(time_out)):
            # Interpolate zonal mean to grid latitude
            f0 = np.interp(lat_out, pN.lat, pN[iforc][:,itime])

            # Repeat the profile everywhere in our domain
            f_out[itime, :, :] = np.tile(f0.reshape(len(lat_out),1), len(lon_out))

        # There should not be any NaNs, but replace in case
        f_out[np.isnan(f_out)] = fillval
        vname = iforc

        sforc(gpath, vname, fname_out, lat_out, lon_out, time_out, f_out)

In [3]:
def stress_cos(yh, mean_amplitude, shift = 0, seasonal_amplitude = 0.1, edge_zero = False):
    y_rescale = (yh-20)/(60-20)
    
    seas_var = 1 + seasonal_amplitude * np.cos(2*np.pi*np.linspace(0,11,12)/12)
    seas_var = np.full((len(yh),12),seas_var)
    
    if edge_zero:
        stress_mean = -mean_amplitude*np.cos(2*np.pi*y_rescale) + shift
        stress_mean = np.full((12,len(yh)), stress_mean).T
        seas_stress = stress_mean * seas_var
        
    else:
        stress_mean = -mean_amplitude*np.cos(2*np.pi*y_rescale)
        stress_mean = np.full((12,len(yh)), stress_mean).T
        seas_stress = stress_mean * seas_var + shift
     
    return seas_stress

def stress_to_wind(stress):
    cd    = 1.8 * 10**(-3) # CD (1.56)  
    rho_a = 1.29       # kg/m3
    
    sign = (stress>0)*2-1 
    wind = np.sqrt(np.abs(stress)/(cd*rho_a))*sign
    
    return wind

In [5]:
fnameLF_out = '_idealized_sin_highres_1yr.nc'
gpath = '../../forcings/idealized_year_cold_NHtemp/gridded/'
forcings = ['uas', 'vas']

# Lat and lon of forcing domain
hres     = 0.1
lat_out  = np.arange(15.5, 70, hres)  # from 15.5 to 69.5
lon_out  = np.arange(300, 350.5, hres)  # from 300 to 350

# Fill value
fillval  = -1e+20  # set fill value (Raphael/Enhui: -1e+20; Elizabeth:-1.e+34)

# Create a 12-month time vector 
time_m = pd.date_range(start = '1990-01-01', periods=12, freq = 'MS')+ pd.DateOffset(days=14)
time_m = np.asarray(time_m.dayofyear, dtype=np.float64)-1
month=np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

In [6]:
stress_highres = stress_cos(lat_out, mean_amplitude = 0.1, seasonal_amplitude = 0.1, shift = 0., edge_zero = True)
wind_highres   = stress_to_wind(stress_highres)

In [7]:
wind = wind_highres

ds_uas = xr.Dataset(data_vars=dict(uas=(["lat", "month"], wind)), coords=dict(lat = lat_out ,month=month))
ds_vas = np.isnan(ds_uas)*1
ds_vas = ds_vas.rename(name_dict={"uas": "vas"})

ds_all = [ds_uas,ds_vas]

In [8]:
save_wind(ds_all, forcings, time_m, lat_out, lon_out, gpath, fnameLF_out)